<a href="https://colab.research.google.com/github/RyanCord/Big-Data-Projeto/blob/main/projeto_de_extensao_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl


In [ ]:
import pandas as pd

file_path = "/content/Dados_Cartório.xlsx"


pdf = pd.read_excel(file_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyspark


In [ ]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.master("local[*]").getOrCreate()


df = spark.createDataFrame(pdf)
df.show(5)

In [ ]:
df.printSchema()


In [ ]:
df.describe().show()



In [ ]:
pdf = df.toPandas()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(data=pdf, x="ESTADO CIVIL", y="PROFISSÃO")
plt.title("Gráfico de Dispersão")
plt.show()

In [ ]:

mean_age_by_profession = pdf.groupby('PROFISSÃO')['DATA NASCIMENTO'].mean().reset_index()
mean_age_by_profession.columns = ['PROFISSÃO', 'Mean_Age']


print(mean_age_by_profession)


In [ ]:
print(mean_age_by_profession)

In [ ]:
pdf['Age'] = pd.to_datetime(pdf['DATA NASCIMENTO']).apply(lambda x: (datetime.datetime.now() - x).days // 365)

mean_age_by_profession = pdf.groupby('PROFISSÃO')['Age'].mean().reset_index()
mean_age_by_profession.columns = ['Profissão', 'Idade_Média']

print(mean_age_by_profession)

In [ ]:
print(mean_age_by_profession)


In [ ]:
df.show()


In [ ]:
from pyspark.sql.functions import when, count


homens_coletados = df.filter(df['SEXO'] == 'Masculino').groupBy('STATUS BIOMETRIA').agg(count('*').alias('Total'))
mulheres_coletadas = df.filter(df['SEXO'] == 'Feminino').groupBy('STATUS BIOMETRIA').agg(count('*').alias('Total'))


homens_coletados = homens_coletados.withColumn('STATUS BIOMETRIA', when(homens_coletados['STATUS BIOMETRIA'].isNull(), 'Não Coletado').otherwise(homens_coletados['STATUS BIOMETRIA']))
mulheres_coletadas = mulheres_coletadas.withColumn('STATUS BIOMETRIA', when(mulheres_coletadas['STATUS BIOMETRIA'].isNull(), 'Não Coletado').otherwise(mulheres_coletadas['STATUS BIOMETRIA']))


print("Homens que coletaram a biometria:")
homens_coletados.show()
print("Mulheres que coletaram a biometria:")
mulheres_coletadas.show()

In [ ]:
import matplotlib.pyplot as plt


labels = ['Coletado', 'Não Coletado']
sizes_homens = homens_coletados.select('Total').rdd.flatMap(lambda x: x).collect()
sizes_mulheres = mulheres_coletadas.select('Total').rdd.flatMap(lambda x: x).collect()


plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.pie(sizes_homens, labels=labels, autopct='%1.1f%%', startangle=140, colors=['lightblue', 'lightgray'])
plt.title("Homens")


plt.subplot(1, 2, 2)
plt.pie(sizes_mulheres, labels=labels, autopct='%1.1f%%', startangle=140, colors=['lightpink', 'lightgray'])
plt.title("Mulheres")

plt.suptitle("Status da Biometria", fontsize=16)


plt.axis('equal')
plt.show()

In [ ]:
from pyspark.sql import functions as F

from pyspark.sql.functions import year, current_date

df_with_age = df.withColumn('IDADE', year(current_date()) - year('DATA NASCIMENTO'))

def calcular_faixa_etaria(idade):
    if idade < 30:
        return '18-29'
    elif 30 <= idade < 40:
        return '30-39'
    elif 40 <= idade < 50:
        return '40-49'
    elif 50 <= idade < 60:
        return '50-59'
    else:
        return '60+'
calc_faixa_etaria_udf = udf(calcular_faixa_etaria, StringType())

df_with_age_range = df_with_age.withColumn('FAIXA_ETARIA', calc_faixa_etaria_udf(df_with_age['IDADE']))


profissoes_por_grupo = df_with_age_range.groupBy('FAIXA_ETARIA', 'SEXO', 'PROFISSÃO').count()

profissoes_mais_comuns = profissoes_por_grupo.orderBy('FAIXA_ETARIA', 'SEXO', F.desc('count')).groupBy('FAIXA_ETARIA', 'SEXO').agg(F.first('PROFISSÃO')).orderBy('FAIXA_ETARIA', 'SEXO')

profissoes_mais_comuns.show()

In [ ]:
import matplotlib.pyplot as plt


profissoes_mais_comuns_pd = profissoes_mais_comuns.toPandas()

for idx, row in profissoes_mais_comuns_pd.iterrows():
    faixa_etaria = row['FAIXA_ETARIA']
    sexo = row['SEXO']
    profissao = row['first(PROFISSÃO)']


    filtro = (df_with_age_range['FAIXA_ETARIA'] == faixa_etaria) & (df_with_age_range['SEXO'] == sexo) & (df_with_age_range['PROFISSÃO'] == profissao)
    total = df_with_age_range.filter(filtro).count()


    plt.figure(figsize=(8, 6))
    plt.pie([total, 100-total], labels=[profissao, 'Outras Profissões'], autopct='%1.1f%%', startangle=140)
    plt.title(f'Profissões mais comuns para {sexo} na faixa etária {faixa_etaria}')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

genero_por_profissao = df.groupBy('PROFISSÃO', 'SEXO').count()

genero_por_profissao_pd = genero_por_profissao.toPandas()

genero_por_profissao_pivot = genero_por_profissao_pd.pivot(index='PROFISSÃO', columns='SEXO', values='count')

plt.figure(figsize=(10, 8))
genero_por_profissao_pivot.plot(kind='pie', subplots=True, layout=(2, 1), autopct='%1.1f%%', startangle=140, legend=False, figsize=(10, 8))
plt.title('Distribuição de Gênero por Profissão')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

distribuicao_genero_estado_civil = df.groupBy('ESTADO CIVIL', 'SEXO').count()

distribuicao_genero_estado_civil_pd = distribuicao_genero_estado_civil.toPandas()

distribuicao_genero_estado_civil_pivot = distribuicao_genero_estado_civil_pd.pivot(index='ESTADO CIVIL', columns='SEXO', values='count')

distribuicao_genero_estado_civil_pivot.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Distribuição de Gênero por Estado Civil')
plt.xlabel('Estado Civil')
plt.ylabel('Contagem')
plt.xticks(rotation=45)
plt.legend(title='Gênero')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


print(df.columns)

# Filtrar os dados para procedimentos realizados no RJ
procedimentos_rj_df = df[df['ESTADO'] == 'RJ']

# Contar o total de registros no RJ
total_procedimentos_rj = procedimentos_rj_df.shape[0]

# Contar o número de pessoas que não são do RJ
procedimentos_nao_rj = procedimentos_rj_df[procedimentos_rj_df['ESTADO'] != 'RJ'].shape[0]

# Calcular a porcentagem
percentual_nao_rj = (procedimentos_nao_rj / total_procedimentos_rj) * 100

# Criar os dados para o gráfico de pizza
labels = ['De fora do RJ', 'Do RJ']
sizes = [percentual_nao_rj, 100 - percentual_nao_rj]
colors = ['#ff9999','#66b3ff']
explode = (0.1, 0)


plt.figure(figsize=(8, 8))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=140)
plt.title('Porcentagem de pessoas que não são do RJ e vêm fazer procedimentos no RJ')
plt.axis('equal')
plt.show()

In [ ]:
plt.show()


In [ ]:
import pandas as pd


df = pd.read_excel("/content/Dados_Cartório.xlsx")


df.head()

In [ ]:
total_pessoas = len(df)
nao_rj = len(df[df['ESTADO'] != 'RJ'])

# Calcular a porcentagem
porcentagem_nao_rj = (nao_rj / total_pessoas) * 100
porcentagem_rj = 100 - porcentagem_nao_rj

print("Porcentagem de pessoas do RJ:", porcentagem_rj)
print("Porcentagem de pessoas de fora do RJ:", porcentagem_nao_rj)

In [ ]:
import matplotlib.pyplot as plt

labels = ['RJ', 'Outros Estados']
sizes = [porcentagem_rj, porcentagem_nao_rj]
colors = ['lightblue', 'lightcoral']

plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.title('Porcentagem de Pessoas do RJ vs. de Fora do RJ')
plt.show()

In [ ]:
sexo_counts = df['SEXO'].value_counts()

plt.figure(figsize=(6, 4))
sexo_counts.plot(kind='bar', color='lightcoral')
plt.title('Distribuição por Sexo')
plt.xlabel('Sexo')
plt.ylabel('Número de Pessoas')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

profissoes_comuns = df['PROFISSÃO'].value_counts().head(5).index.tolist()

df_comuns = df[df['PROFISSÃO'].isin(profissoes_comuns)]

estado_civil_counts = df_comuns.groupby(['PROFISSÃO', 'ESTADO CIVIL']).size().unstack(fill_value=0)

estado_civil_counts.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Estado Civil por Profissão (Top 5 Profissões Mais Comuns)')
plt.xlabel('Profissão')
plt.ylabel('Número de Pessoas')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Estado Civil')
plt.tight_layout()
plt.show()